# FIN-413 Project

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime, timedelta
import warnings
import scipy
from scipy import stats
from scipy.optimize import minimize
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import gaussian_kde
from tqdm import tqdm

warnings.filterwarnings("ignore", category=FutureWarning)


# Useful function to display dataframe + display shape
def disp(df, n=5, title=None):
    if title:
        print(title)
    display(df.head(n))
    print(df.shape)

def save_large_tables(dataframe, path):
    c = dataframe.columns.to_list()
    dataframe[c[:len(c)//2]].to_latex(f"{path}_part_1.tex")
    dataframe[c[len(c)//2:]].to_latex(f"{path}_part_2.tex")

## Part 1 - Exploratory Data Analysis

### Preliminary considerations

In [ ]:
data = pd.read_csv("data.csv", skiprows=[0])

# Parse the 'time' column to datetime if it's not already in that format
data['time'] = pd.to_datetime(data['time'], errors='coerce')

# Convert all other columns to numeric values, assuming they represent returns
for col in data.columns[1:]:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Set the index
data.set_index("time", drop = True, inplace = True)

# Data overview
print("Data overview")
display(data)
save_large_tables(data, "tables/part1_returns_raw")
# data.to_latex("tables/part1_returns_raw.tex")

# Data description
print("Data description")
display(data.info())

# Create list of the names of the crypto / indices
cryptos = ['ADA', 'BCH', 'BTC', 'DOGE', 'ETH', 'LINK', 'LTC', 'MANA', 'XLM', 'XRP']
indices = ['SPXT', 'XCMP', 'USSOC', 'VIX']

# Important dates
datePP = datetime.strptime("2021/09/11", "%Y/%m/%d")
dateTr = datetime.strptime("2022/11/21", "%Y/%m/%d")
dateRec = datetime.strptime("2024/03/07", "%Y/%m/%d")

In [ ]:
# Descriptive statistics
print("Descriptive statistics")
descriptive_stats = data.describe()
display(descriptive_stats)
# descriptive_stats.to_latex("tables/part1_descriptive_stats.tex")
save_large_tables(descriptive_stats, "tables/part1_descriptive_stats")


In [ ]:
# Heatmap: correlation over the assets
plt.figure(figsize=(14, 10))
correlation_matrix = data.corr() 
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.savefig("figures/part1_correlation_heatmap")
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Rolling correlations for crypto and traditional assets
window_size = 30  # 30 days rolling window

# Iterate over each cryptocurrency
for crypto in cryptos: # ['ADA', 'BCH', 'BTC', 'DOGE', 'ETH', 'LINK', 'LTC', 'MANA', 'XLM', 'XRP']
    
    fig = go.Figure()
    
    # Iterate over each traditional asset
    for trad in indices: # ['SPXT', 'XCMP', 'USSOC', 'VIX']
        # Calculate rolling correlation
        rolling_corr = data[crypto].rolling(window=window_size).corr(data[trad])
        
        # Ensure the index is a flat array for plotting
        rolling_corr_index = rolling_corr.index.values.flatten()
        fig.add_trace(go.Scatter(x = rolling_corr_index, y=rolling_corr.values, mode='lines', name=f'{crypto} vs {trad}'))
    
    fig.update_layout(title=f'Rolling Correlation of {crypto} with Traditional Assets',
                 xaxis_title='Time',
                 yaxis_title='Correlation')
    fig.show()

In [ ]:
# Individual density of the assets

# Create generator to have the name of all securities
def give_col(data):
    for col in data.columns:
        yield col

def plot_sub_hist(df, title="Distribution of returns of the different securities", show=True, rows_nb = 7, cols_nb = 2, save=False, save_title="figures/part1b_distribution_returns"):
    column_generator = give_col(df)


    rows = rows_nb # Number of rows
    cols = cols_nb # Number of columns

    fig = make_subplots(rows=rows, cols=cols, subplot_titles=list(give_col(df)))
    # Loop through data and add traces
    for col in range(cols):
        col += 1
        for row in range(rows):
            row += 1
            sec = next(column_generator)
            trace = go.Histogram(
                x=df[sec], 
                nbinsx=300,
                name = f"{sec}")
            fig.append_trace(trace, row, col)

    fig.update_layout(
        height=rows*200, 
        width=1500, 
        title_text=title)
    
    if show:
        fig.show()


plot_sub_hist(data)

In [ ]:
# Marginal density plots for each asset
plt.figure(figsize=(14, 7))
for column in data.columns[1:]:         # Skipping 'time' column
    sns.kdeplot(data[column].dropna(), fill=True, label=column)  # Drop NA values for kde plot
plt.savefig("figures/part1_marginal_density_plot.png")
plt.title('Marginal Density Plots for All Assets')
plt.legend()
plt.show()

fig = px.histogram(
    data_frame=data, 
    histnorm='density')
fig.show()

In [ ]:
# Not sure that this is relevant - Could be removed ? # Agu: It is a little bit unreadable like this. Maybe we can separate the plots? 
fig = px.scatter(
    data,
    y = cryptos, x = data.index,
    color_continuous_scale='picnic',
    width = 1600, height = 600,
    title = 'Daily values of cryptos against USD',
)

fig.show()

### Question 1.a) Log returns

*__TO BE REMOVED OR UPDATED WITH LATEST VERSION FROM THE REPORT__*

For this analysis, log returns should be used. The marginal distributions of crypto assets typically exhibit high kurtosis and skewness, suggesting a non-normal distribution. Log returns, which normalize and stabilize variance, are more suitable for the statistical models used in risk-based portfolio optimization, particularly when dealing with the high volatility of cryptocurrencies. Furthermore, cryptocurrency prices can fluctuate rapidly since they are traded continuously. Log returns, which consider compounding effects, provide a more accurate representation of returns. This choice aligns with Meucci's insights on the appropriateness of log returns for multiperiod investment horizons and assets with volatile behaviors.

In [ ]:
# Tranform values into log
data_log = data.apply(lambda x: np.log(x+1))
# data_log.to_latex("tables/part1a_log_returns.tex")
save_large_tables(data_log, "tables/part1a_log_returns")
print("Log returns")
disp(data)
print("Missing data:")
disp(data.isna().sum(), 15)

In [ ]:
# Individual distribution of log returns
plot_sub_hist(data_log, title = "Distribution of log returns")

### Question 1.b) Outliers

#### Using linear returns - for information

In [ ]:
# Identify potential outliers
potential_outliers = data[(np.abs(data - data.mean()) > (3 * data.std())).any(axis=1)]
# potential_outliers.to_latex("tables/part1b_potential_outliers.tex")
save_large_tables(potential_outliers, "tables/part1b_potential_outliers")
disp(potential_outliers)

In [ ]:
# Calculate the IQR for each column except 'time' and identify outliers
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1
outliers = (data < (Q1 - 3 * IQR)) | (data > (Q3 + 3 * IQR))

# Calculate the total number of outliers identified
total_outliers = outliers.sum().sum()  # Summing up all True values for outliers

# Handle outliers: Replace them with NaN for columns other than 'time'
data_cleaned = data.copy()
data_cleaned = data.mask(outliers)
data_cleaned.to_latex("tables/part1b_data_clean.tex")
save_large_tables(data_cleaned,"tables/part1b_data_clean")

# Fill NaN values using forward fill, then backward fill for any remaining at the start
data_cleaned.fillna(method='ffill', inplace=True)
data_cleaned.fillna(method='bfill', inplace=True)

# Print the total number of outliers
print("Total number of outliers identified:", total_outliers)

disp(outliers)

print("\nClean data")
disp(data_cleaned)

*__TO BE REMOVED OR UPDATED WITH LATEST VERSION FROM THE REPORT__*

We conducted outlier detection using two distinct statistical methods due to the unique characteristics of financial data, especially in highly volatile markets like those of cryptocurrencies.

Standard Deviation Method: The first approach involved identifying outliers as those data points lying beyond three standard deviations from the mean. While commonly used in many fields for outlier detection, this method proved overly sensitive for our dataset. It flagged a large number of data points as outliers, which upon further examination, were within plausible ranges for cryptocurrency returns. This high sensitivity is attributed to the large standard deviations characteristic of these markets, suggesting that the method might not be suitable for data with high volatility.

Interquartile Range (IQR) Method: Given the limitations observed with the standard deviation approach, we shifted to the IQR method. The IQR technique is less affected by extreme values and provides a more robust measure for identifying outliers in skewed distributions. Initially, a commonly used multiplier of 1.5 was applied to the IQR to define the outliers, which still resulted in a high number of outliers. Adjusting the multiplier to 3 significantly reduced the count to 1361 outliers, offering a more reasonable quantification (relatively to our original dataset size) that reflects true anomalous behaviors without discarding legitimate data variations.

Handling Identified Outliers: After identifying outliers with the adjusted IQR method, we handled these by initially replacing them with NaN values to prevent any distortion in the subsequent analysis. To maintain the continuity and integrity of our time series data, these NaN values were then addressed using forward filling (ffill), which propagates the last valid observation forward. Where forward filling was insufficient (e.g., at the start of the data series), backward filling (bfill) was employed to ensure no data point was left unfilled. This approach helps in preserving the temporal structure of the data while minimizing the impact of extreme value fluctuations.



#### Using log returns

In [ ]:
# Calculate log returns for all assets
data_log_returns = np.log1p(data)

In [ ]:
# Identify potential outliers
potential_outliers_logreturns = data_log_returns[(np.abs(data_log_returns - data_log_returns.mean()) > (3 * data_log_returns.std())).any(axis=1)]
# potential_outliers.to_latex("tables/part1b_potential_outliers.tex")
save_large_tables(potential_outliers_logreturns, "tables/part1b_potential_outliers_std_method_logreturn")
disp(potential_outliers_logreturns)

In [ ]:
# Calculate the IQR for log returns
Q1_log = data_log_returns.quantile(0.25)
Q3_log = data_log_returns.quantile(0.75)
IQR_log = Q3_log - Q1_log

# Identify outliers using the IQR method for log returns
outliers_log = (data_log_returns < (Q1_log - 3 * IQR_log)) | (data_log_returns > (Q3_log + 3 * IQR_log))

# Calculate the total number of outliers
total_outliers_log = outliers_log.sum().sum()

# Handle outliers by replacing them with NaN and then forward-filling
data_log_returns_masked = data_log_returns.mask(outliers_log)
data_log_returns_filled = data_log_returns_masked.fillna(method='ffill').fillna(method='bfill')
data_log_returns_filled.to_latex("tables/part1b_data_clean_logreturns.tex")
save_large_tables(data_log_returns_filled, "tables/part1b_data_clean_logreturns")

print("Total number of outliers identified:", total_outliers_log)
disp(data_log_returns_filled)


In [ ]:
# First, let's rescale the marginal density plots with log returns
fig = px.histogram(
    data_frame=data_log_returns, 
    histnorm='density')
fig.update_layout(title=f'Rescaled Marginal Density Plots for Log Returns',
                 xaxis_title='Log Returns',
                 yaxis_title='Density')
fig.show()
fig.write_image("figures/part1b_rescaled_marginal_density_logreturns.png")



# Now, we'll conduct statistical tests for skewness and kurtosis to quantify the shape of the distribution.
skewness = data_log_returns.skew().sort_values()
skewness.to_latex("tables/part1b_skewness.tex")
kurtosis = data_log_returns.kurtosis().sort_values()
kurtosis.to_latex("tables/part1b_kurtosis.tex")


# Display skewness and kurtosis values
print("Skewness of Log Returns:")
print(skewness)
print("\nKurtosis of Log Returns:")
print(kurtosis)

# Next, we'll create a Q-Q plot for one of the assets to visually inspect its distribution against a normal distribution.
# Here we use BTC as an example. we would repeat this for other assets or iterate over all.
plt.figure(figsize=(8, 8))
stats.probplot(data_log_returns['BTC'].dropna(), dist="norm", plot=plt)
plt.savefig("figures/part1b_QQ_plot_BTC_log")
plt.title('Q-Q Plot for BTC Log Returns')
plt.show()

# Lastly, we'll plot a box plot to visualize the spread and outliers in the log returns.
plt.figure(figsize=(14, 7))
sns.boxplot(data=data_log_returns)
plt.xlabel('Assets')
plt.ylabel('Log Returns')
plt.xticks(rotation=45)
plt.savefig("figures/part1b_box_plot")
plt.title('Box Plot for Log Returns')
plt.show()


### Question 1.c) Total returns

*__TO BE REMOVED OR UPDATED WITH LATEST VERSION FROM THE REPORT__*

Does it Matter in General?
Yes, significantly because of 2 aspects:

- Accuracy in Returns: Total return indices, which include dividends, provide a fuller picture of the true returns an investor would receive, as they reflect both price changes and income from dividends.
- Benchmarking and Relevance: They offer a more realistic benchmark for assessing investment performance and are particularly relevant for investors who reinvest dividends.

For the Risk-Based Portfolio Optimizations in This Study?
Indeed, it is also crucial:

- Impact on Risk and Reward: The inclusion of dividends affects both the risk and return calculations, altering the risk-return profile used in portfolio optimizations.
- Influence on Diversification: These indices might show different behaviors compared to price-only indices, particularly affecting diversification strategies due to changes in asset correlations.
- Optimization Outcomes: Total return indices can shift the efficient frontier upward, leading to different optimal portfolio compositions and weightings.

In a nutshell, using total return indices like SPXT and XCMP is essential for realistic and effective portfolio management, especially in a study focusing on risk-based optimization. This approach ensures that all components of returns are considered, facilitating more accurate decision-making in portfolio construction and management.

### Question 1.d) Weekend activity

*__TO BE REMOVED OR UPDATED WITH LATEST VERSION FROM THE REPORT__*

Observation and Impact:
For traditional assets like SPXT and XCMP, zero returns over weekends are noted because these markets are closed, whereas cryptocurrencies trade 24/7, leading to non-zero returns every day. This discrepancy can distort correlation calculations between traditional and crypto assets, as weekends introduce artificial stability into the returns of traditional assets.

This is a problem for statistical analyses, especially when calculating correlations or conducting any comparative time series analysis. The zero returns can artificially lower the volatility and skew the correlation metrics between traditional and crypto assets.

Suggested Solution:

- Synchronize the Data: One approach is to adjust the dataset so that returns for traditional assets are only calculated on days when those markets are open (i.e., exclude weekends). Alternatively, calculate weekly returns instead of daily returns to ensure comparability across all assets.
- Use Imputation: For correlation purposes and other time-sensitive analyses, impute weekend returns for traditional assets using interpolation or forward-filling from the nearest non-zero data point. This maintains data continuity without introducing significant bias.
This methodological adjustment helps align the data temporally, providing more accurate and meaningful analytical outcomes, particularly in mixed-asset portfolios like those including both traditional and crypto assets.

## Part 2

In [ ]:
# To make sure we have correct data
DATA_LOG_RETURN = data_log_returns_filled
RETURNS_PP = data_cleaned.copy().loc[:datePP]
RETURNS_TR = data_cleaned.copy().loc[datePP:dateTr]
# display(RETURNS_PP.mean())
# display(RETURNS_TR.mean())

synthetic_pricesPP = (1 + RETURNS_PP).cumprod()
synthetic_pricesTR = (1 + RETURNS_TR).cumprod()

display(synthetic_pricesPP)

synthetic_pricesPP.plot(figsize=(16,6))
plt.show()
display(synthetic_pricesTR)
synthetic_pricesTR.plot(figsize=(16,6))
plt.show()

### Question 2.a) Equally weigthed portfolio

In [ ]:
# Weights
weights_EW = 1 / len(DATA_LOG_RETURN.columns)
weights_EW_list = np.array([weights_EW for i in range(len(DATA_LOG_RETURN.columns))])

In [ ]:
portfolio_EW_log = DATA_LOG_RETURN.copy()
portfolio_EW_log["Return_EW"] = portfolio_EW_log.apply(lambda row: row.sum(), axis=1) * weights_EW
print("Marginal contribution EW portfolio")
disp(portfolio_EW_log)
portfolio_EW_log.to_latex("tables/part2a_EW_marginal_contribution.tex")

In [ ]:
# Expected return and std EW portfolio
expected_returns_PP = weights_EW_list @ np.array(RETURNS_PP.mean().to_list()).T
expected_returns_TR = weights_EW_list @ np.array(RETURNS_TR.mean().to_list()).T

variance_EW_PP = (weights_EW_list @ RETURNS_PP.cov() @ weights_EW_list.T)
variance_EW_TR = (weights_EW_list @ RETURNS_TR.cov() @ weights_EW_list.T)


# print("Expected return:\t", round(expected_returns, 4))
# print("Standard deviation:\t", round(variance_EW**0.5, 4))

print(f"Equally weighted portfolio as of datePP")
print(f"  - Expected return: \t {round(expected_returns_PP, 4)}%")
print(f"  - Volatility: \t {round(variance_EW_PP ** 0.5, 4)}%")
print()
print(f"Equally weighted portfolio as of dateTr")
print(f"  - Expected return: \t {round(expected_returns_TR, 4)}%")
print(f"  - Volatility: \t {round(variance_EW_TR ** 0.5, 4)}%")

In [ ]:
# Plot the returns of the portfolio EW
fig = go.Figure()

fig.add_trace(go.Scatter(x = portfolio_EW_log.loc[dateTr:dateRec].index, 
                 y=portfolio_EW_log.loc[dateTr:dateRec].Return_EW, 
                 mode='lines', 
                 name=f'Before {datetime.strftime(dateRec,"%Y/%m/%d")} (dateRec)',
                 ))

fig.add_trace(go.Scatter(x = portfolio_EW_log.loc[datePP:dateTr].index, 
                 y=portfolio_EW_log.loc[datePP:dateTr].Return_EW, 
                 mode='lines', 
                 name=f'Before {datetime.strftime(dateTr,"%Y/%m/%d")} (dateTr)',
                 ))

fig.add_trace(go.Scatter(x = portfolio_EW_log.loc[:datePP].index, 
                 y=portfolio_EW_log.loc[:datePP].Return_EW, 
                 mode='lines', 
                 name=f'Before {datetime.strftime(datePP,"%Y/%m/%d")} (datePP)',
                 ))

fig.update_layout(
    # title=f'Return of an equally weighted portfolio of the 14 assets (including cryptos and traditional assets)',
    xaxis_title='Time',
    yaxis_title='Return EW portolio'
    )

fig.write_image("figures/part2a_ew_portfolio.png") # Requires to install ! pip install -U kaleido

fig.update_layout(
    title=f'Daily log-return equally weighted portfolio of the 14 assets (including cryptos and traditional assets)'
    )

fig.show()

In [ ]:
# Same plot using Matplotlib for the report

# Plot the returns of the portfolio EW
plt.figure(figsize=(20, 6))

plt.plot(portfolio_EW_log.loc[dateTr:dateRec].index, 
         portfolio_EW_log.loc[dateTr:dateRec].Return_EW, 
         label=f'Before {datetime.strftime(dateRec,"%Y/%m/%d")} (dateRec)', 
         color='black')

plt.plot(portfolio_EW_log.loc[datePP:dateTr].index, 
         portfolio_EW_log.loc[datePP:dateTr].Return_EW, 
         label=f'Before {datetime.strftime(dateTr,"%Y/%m/%d")} (dateTr)', 
         color='dimgray')

plt.plot(portfolio_EW_log.loc[:datePP].index, 
         portfolio_EW_log.loc[:datePP].Return_EW, 
         label=f'Before {datetime.strftime(datePP,"%Y/%m/%d")} (datePP)', 
         color='darkblue')


plt.xlabel('Time')
plt.ylabel('Return EW portfolio')
plt.savefig("figures/part2a_ew_portfolio.png")
plt.title('Daily log-return equally weighted portfolio of the 14 assets (including cryptos and traditional assets)')
plt.legend()
plt.show()



### Question 2.b) Covariance matrix

#### Using information from Lopez de Prado 2016.

In [ ]:
# Prado 2016: need 0.5*N*(N-1)IID observations to get non singular variance covariance matrix of size N. Here: N=14
N = len(DATA_LOG_RETURN.columns)
size = int(round(0.5*N*(N-1), 0))
print(f"Use {size} days of data.")

In [ ]:
print(f"Variance covariance matrix - on {datetime.strftime(datePP, '%Y/%m/%d')} (dateTr)")
cov_matrix_datePP = DATA_LOG_RETURN.loc[datePP-timedelta(size):datePP].cov()
cov_matrix_datePP.to_latex("tables/part2b_cov_matrix_datePP.tex")
save_large_tables(cov_matrix_datePP, "tables/part2b_cov_matrix_datePP")
display(cov_matrix_datePP)

print(f"Variance covariance matrix - on {datetime.strftime(dateTr, '%Y/%m/%d')} (datePP)")
cov_matrix_dateTr = DATA_LOG_RETURN.loc[dateTr-timedelta(size):dateTr].cov()
cov_matrix_dateTr.to_latex("tables/part2b_cov_matrix_dateTr.tex")
save_large_tables(cov_matrix_dateTr, "tables/part2b_cov_matrix_dateTr")
display(cov_matrix_dateTr)

In [ ]:
# Test: might be totally wrong to do that: plot a heat map of the difference of covariance between the two dates
diff_covariance = round(cov_matrix_datePP - cov_matrix_dateTr, 4)

# Before datePP
plt.figure(figsize=(14, 10))
sns.heatmap(diff_covariance, annot=True, cmap='coolwarm')
plt.savefig("figures/part2b_heatmap_difference_covariance")
plt.title('Covariance difference Heatmap')
plt.show()

### Question 2.c) Clean the covariance matrix

In [ ]:
#Need to get the threshold to clean
def determine_threshold(cov_matrix):

    eigenvalues, _ = np.linalg.eigh(cov_matrix)
    #Sort
    eigenvalues_sorted = np.sort(eigenvalues)[::-1]
    #Set threshold at value where eigenvalues appear to level off
    threshold_index = np.where(eigenvalues_sorted < 0.25)[0][0]  
    # print(threshold_index)
    threshold = eigenvalues_sorted[threshold_index]

    return threshold

In [ ]:
# Cleaning the covariance matrix
def clean_covariance_matrix(cov_matrix, threshold):

    volatilities = np.sqrt(np.diag(cov_matrix))
    correlation_matrix = cov_matrix / np.outer(volatilities, volatilities)

    # Spectral decomposition to obtain eigenvalues and eigenvectors of correlation matrix
    eigenvalues, eigenvectors = np.linalg.eigh(correlation_matrix)

    # print(eigenvalues)

    # Clip the eigenvalues
    clipped_eigenvalues = np.maximum(eigenvalues, threshold)
    # print(clipped_eigenvalues)
    diagonal_matrix = np.diag(clipped_eigenvalues)
    cleaned_corr = eigenvectors @ diagonal_matrix @ eigenvectors.T

    # Reconstruct the covariance matrix 
    cleaned_covariance_matrix= np.outer(volatilities, volatilities) * cleaned_corr
    
    return cleaned_covariance_matrix

In [ ]:
#Date PP
volatilities_PP = np.sqrt(np.diag(cov_matrix_datePP))
correlation_matrix_PP = cov_matrix_datePP / np.outer(volatilities_PP, volatilities_PP)

thres_PP = determine_threshold(correlation_matrix_PP)
print('Threshold for date PP:', thres_PP)

cleaned_covariance_matrix_datePP = clean_covariance_matrix(cov_matrix_datePP, thres_PP)
cleaned_covariance_matrix_datePP_df = pd.DataFrame(cleaned_covariance_matrix_datePP, index=cov_matrix_datePP.index, columns=cov_matrix_datePP.columns)

print('\nThe difference:')
display(cov_matrix_datePP-cleaned_covariance_matrix_datePP) # there are differences

#Date Tr
volatilities_Tr = np.sqrt(np.diag(cov_matrix_dateTr))
correlation_matrix_Tr = cov_matrix_dateTr / np.outer(volatilities_Tr, volatilities_Tr)

thres_Tr = determine_threshold(correlation_matrix_Tr)
print('\nThreshold for date Tr:', thres_Tr)

cleaned_covariance_matrix_dateTr = clean_covariance_matrix(cov_matrix_dateTr, thres_Tr)
cleaned_covariance_matrix_dateTr_df = pd.DataFrame(cleaned_covariance_matrix_dateTr, index=cov_matrix_dateTr.index, columns=cov_matrix_dateTr.columns)

print('The difference:')
display(cov_matrix_dateTr-cleaned_covariance_matrix_dateTr)

# Save tables
cleaned_covariance_matrix_datePP_df.to_latex("tables/part2c_cleaned_covariance_matrix_datePP.tex")
cleaned_covariance_matrix_dateTr_df.to_latex("tables/part2c_cleaned_covariance_matrix_dateTr.tex")
save_large_tables(cleaned_covariance_matrix_datePP_df, "tables/part2c_cleaned_covariance_matrix_datePP")    # For the report
save_large_tables(cleaned_covariance_matrix_dateTr_df, "tables/part2c_cleaned_covariance_matrix_dateTr")    # Report friendly

In [ ]:
# Plot 3D visualizations of raw and cleaned covariance matrices at datePP

fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')
X1, Y1 = np.meshgrid(np.arange(len(cov_matrix_datePP)), np.arange(len(cov_matrix_datePP)))
ax1.plot_surface(X1, Y1, cov_matrix_datePP, cmap='viridis')
ax1.set_xlabel('Asset Index')
ax1.set_ylabel('Asset Index')
ax1.set_zlabel('Covariance')
fig1.savefig("figures/part2c_raw_cov_matrix_datePP")
ax1.set_title('Raw Covariance Matrix (datePP)')

fig2 = plt.figure(figsize=(8, 6))
ax2 = fig2.add_subplot(111, projection='3d')
X2, Y2 = np.meshgrid(np.arange(len(cleaned_covariance_matrix_datePP)), np.arange(len(cleaned_covariance_matrix_datePP)))
ax2.plot_surface(X2, Y2, cleaned_covariance_matrix_datePP, cmap='viridis')
ax2.set_xlabel('Asset Index')
ax2.set_ylabel('Asset Index')
ax2.set_zlabel('Covariance')
fig2.savefig("figures/part2c_cleaned_cov_matrix_datePP")
ax2.set_title('Cleaned Covariance Matrix (datePP)')

# Plot 3D visualizations of raw and cleaned covariance matrices at dateTr
fig3 = plt.figure(figsize=(8, 6))
ax3 = fig3.add_subplot(111, projection='3d')
X3, Y3 = np.meshgrid(np.arange(len(cov_matrix_dateTr)), np.arange(len(cov_matrix_dateTr)))
ax3.plot_surface(X3, Y3, cov_matrix_dateTr, cmap='viridis')
ax3.set_xlabel('Asset Index')
ax3.set_ylabel('Asset Index')
ax3.set_zlabel('Covariance')
fig3.savefig("figures/part2c_raw_cov_matrix_dateTr")
ax3.set_title('Raw Covariance Matrix (dateTr)')

fig4 = plt.figure(figsize=(8, 6))
ax4 = fig4.add_subplot(111, projection='3d')
X4, Y4 = np.meshgrid(np.arange(len(cleaned_covariance_matrix_dateTr)), np.arange(len(cleaned_covariance_matrix_dateTr)))
ax4.plot_surface(X4, Y4, cleaned_covariance_matrix_dateTr, cmap='viridis')
ax4.set_xlabel('Asset Index')
ax4.set_ylabel('Asset Index')
ax4.set_zlabel('Covariance')
fig4.savefig("figures/part2c_cleaned_cov_matrix_dateTr")
ax4.set_title('Cleaned Covariance Matrix (dateTr)')

plt.tight_layout()
plt.show()


In [ ]:

def plot_eigen_spectrum(eigenvalues, title):
    density = gaussian_kde(eigenvalues)
    xs = np.linspace(min(eigenvalues), max(eigenvalues), 1000)
    plt.figure(figsize=(6, 4))
    plt.plot(xs, density(xs), label='Eigen-spectrum', linewidth=2)
    plt.title(title, fontsize=12)
    plt.xlabel('Eigenvalue', fontsize=10)
    plt.ylabel('Density', fontsize=10)
    plt.legend(fontsize=10)
    plt.grid(True)
    plt.show()

def compute_condition_number(cov_matrix):
    eigenvalues = np.linalg.eigvalsh(cov_matrix)
    max_eigenvalue = np.max(eigenvalues)
    min_eigenvalue = np.min(eigenvalues)
    condition_number = max_eigenvalue / min_eigenvalue
    return condition_number

In [ ]:
# Plot kernel density estimates of eigen-spectra
plot_eigen_spectrum_PP = plot_eigen_spectrum(np.linalg.eigvalsh(cov_matrix_datePP), 'Eigen-spectrum of Raw Covariance Matrix (datePP)')
plot_eigen_spectrum_PP2 = plot_eigen_spectrum(np.linalg.eigvalsh(cleaned_covariance_matrix_datePP), 'Eigen-spectrum of Cleaned Covariance Matrix (datePP)')
plot_eigen_spectrum_Tr = plot_eigen_spectrum(np.linalg.eigvalsh(cov_matrix_dateTr), 'Eigen-spectrum of Raw Covariance Matrix (dateTr)')
plot_eigen_spectrum_Tr2 = plot_eigen_spectrum(np.linalg.eigvalsh(cleaned_covariance_matrix_dateTr), 'Eigen-spectrum of Cleaned Covariance Matrix (dateTr)')


# Compare condition numbers
raw_condition_number_PP = compute_condition_number(cov_matrix_datePP)
cleaned_condition_number_PP = compute_condition_number(cleaned_covariance_matrix_datePP)
raw_condition_number_Tr = compute_condition_number(cov_matrix_dateTr)
cleaned_condition_number_Tr = compute_condition_number(cleaned_covariance_matrix_dateTr)

print("Condition number of Raw Covariance Matrix (datePP):", raw_condition_number_PP)
print("Condition number of Cleaned Covariance Matrix (datePP):", cleaned_condition_number_PP)
print("Condition number of Raw Covariance Matrix (dateTr):", raw_condition_number_Tr)
print("Condition number of Cleaned Covariance Matrix (dateTr):", cleaned_condition_number_Tr)

### Question 2.d) Euler risk contribution structures

The Euler risk contribution method is a way to decompose the total risk of a portfolio into the risk contributed by each individual asset within the portfolio. It's based on the first-order Taylor expansion of the portfolio's total risk with respect to changes in the weights of individual assets.

The total risk of a portfolio is typically measured by its standard deviation or variance. This quantifies how much the portfolio's value is expected to fluctuate.

The partial derivatives of the portfolio's standard deviation or variance with respect to the weights of individual assets quantify the sensitivity of the portfolio's risk to changes in the weights of those assets. These derivatives tell us how much the portfolio's risk would change if we adjust the weight of a particular asset while keeping the weights of other assets constant.

To calculate the risk contribution of each asset, we multiply the weight of each asset by its respective partial derivative. This gives us the contribution of each asset to the total risk of the portfolio.

The Herfindahl index of the risk contributions provides a single number that summarizes the distribution of risk within the portfolio. It's calculated by summing the squares of the individual asset risk contributions and then dividing by the square of the portfolio standard deviation. A higher Herfindahl index indicates that risk is more concentrated in a few assets, while a lower index indicates more evenly distributed risk.

In [ ]:
def calculate_euler_risk_contributions(weights, cov_matrix):
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    partial_sigma_partial_w = (cov_matrix @ weights) / portfolio_std
    euler_risk_contributions = weights * partial_sigma_partial_w
    return euler_risk_contributions

def calculate_herfindahl_index(euler_risk_contributions):
    return np.sum(np.square(euler_risk_contributions)) / np.square(np.sum(euler_risk_contributions))

In [ ]:
# Each of the 14 assets in the portfolio is allocated an equal weight of 1/14 
weights_date_PP = np.array([1/14] * 14)  
weights_date_Tr = np.array([1/14] * 14)


# Calculate Euler risk contributions for both portfolios at both dates using raw sample covariances
euler_rc_date_PP_raw = calculate_euler_risk_contributions(weights_date_PP, cov_matrix_datePP)
euler_rc_date_Tr_raw = calculate_euler_risk_contributions(weights_date_Tr, cov_matrix_dateTr)


# Calculate Euler risk contributions for both portfolios at both dates using cleaned covariances
euler_rc_date_PP_cleaned = calculate_euler_risk_contributions(weights_date_PP, cleaned_covariance_matrix_datePP)
euler_rc_date_Tr_cleaned = calculate_euler_risk_contributions(weights_date_Tr, cleaned_covariance_matrix_dateTr)


# Calculate Herfindahl index for each portfolio and each estimator
herfindahl_date_PP_raw = calculate_herfindahl_index(euler_rc_date_PP_raw)
herfindahl_date_Tr_raw = calculate_herfindahl_index(euler_rc_date_Tr_raw)
herfindahl_date_PP_cleaned = calculate_herfindahl_index(euler_rc_date_PP_cleaned)
herfindahl_date_Tr_cleaned = calculate_herfindahl_index(euler_rc_date_Tr_cleaned)

print(herfindahl_date_PP_raw)
print(herfindahl_date_Tr_raw)
print(herfindahl_date_PP_cleaned)
print(herfindahl_date_Tr_cleaned)



### Question 2.e) Diversification distribution of the portfolios

In [ ]:
def principal_component_decomposition(correlation_matrix):
    # Compute the eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(correlation_matrix)

    # Sort the eigenvalues in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues_sorted = eigenvalues[sorted_indices]
    eigenvectors_sorted = eigenvectors[:, sorted_indices]

    # The diagonal matrix Delta containing the eigenvalues
    A= np.diag(eigenvalues_sorted)

    # E is the matrix of eigenvectors corresponding to the sorted eigenvalues
    E = eigenvectors_sorted
    return(E,A,eigenvalues_sorted)

def diversification_distribution(weights,eigenvalues):
    var=0
    l=len(eigenvalues)
    List_probability_distribution=[]
    for i in range(l):
        var+=(weights[i]**2)*eigenvalues[i]
    for i in range(l):
        List_probability_distribution.append((weights[i]**2)*eigenvalues[i]/var)
    return(List_probability_distribution)

def Effective_Number_of_Bets(N,K,weights,eigenvalues):
    S=0
    L=diversification_distribution(weights,eigenvalues)
    for i in range(K-1,N):
        S+=L[i]*np.log(L[i])
    return(np.exp(-S),L)

In [ ]:
## 1. Diversification Distribution Date_PP_raw
eigenvalues_sorted_PP_raw = principal_component_decomposition(cov_matrix_datePP)[2]
l = len(eigenvalues_sorted_PP_raw)
weights=[weights_EW for i in range(l)]
Effective_Number_of_Bets_PP_raw = Effective_Number_of_Bets(l, 1, weights, eigenvalues_sorted_PP_raw)[0]
data_plot_PP_raw = {'x': [i for i in range(1, l+1)],'y': Effective_Number_of_Bets(l, 1, weights, eigenvalues_sorted_PP_raw)[1]}
df_plot_PP_raw = pd.DataFrame(data_plot_PP_raw)
df_plot_PP_raw
# Create the plot
sns.set_theme()  # Optional: applies the default seaborn theme styling
plt.figure(figsize=(10, 6))  # Set the figure size
barplot = sns.barplot(x='x', y='y', data=df_plot_PP_raw, color='red', saturation=0.7)
# Adding the dashed line at y=1
plt.axhline(1, color='black', linestyle='--')
for index, row in df_plot_PP_raw.iterrows():
    barplot.text(row['x'] - 1, row['y'], f'{row["y"]:.2f}', color='black', ha='center', va='bottom')
# Set labels (optional)
plt.title('Diversification Distribution Date_PP_raw')
plt.xlabel('principal portfolios')
plt.ylabel('diversification distribution')

## 2. Diversification Distribution Date_Tr_raw
eigenvalues_sorted_Tr_raw=principal_component_decomposition(cov_matrix_dateTr)[2]
Effective_Number_of_Bets_Tr_raw=Effective_Number_of_Bets(l,1,weights,eigenvalues_sorted_Tr_raw)[0]
data_plot_Tr_raw = {'x': [i for i in range(1,l+1)],'y': Effective_Number_of_Bets(l,1,weights,eigenvalues_sorted_Tr_raw)[1]}
df_plot_Tr_raw= pd.DataFrame(data_plot_Tr_raw)
# Create the plot
sns.set_theme()  # Optional: applies the default seaborn theme styling
plt.figure(figsize=(10, 6))  # Set the figure size
barplot = sns.barplot(x='x', y='y', data=df_plot_Tr_raw, color='red', saturation=0.7)
# Adding the dashed line at y=1
plt.axhline(1, color='black', linestyle='--')
for index, row in df_plot_Tr_raw.iterrows():
    barplot.text(row['x'] - 1, row['y'], f'{row["y"]:.2f}', color='black', ha='center', va='bottom')
# Set labels (optional)
plt.title('Diversification Distribution Date_Tr_raw')
plt.xlabel('principal portfolios')
plt.ylabel('diversification distribution')

## 3. Diversification Distribution Date_PP_cleaned
eigenvalues_sorted_PP_cleaned=principal_component_decomposition(cleaned_covariance_matrix_datePP)[2]
l=len(eigenvalues_sorted_PP_cleaned)
weights=[weights_EW for i in range(l)]
Effective_Number_of_Bets_PP_cleaned=Effective_Number_of_Bets(l,1,weights,eigenvalues_sorted_PP_cleaned)[0]
data_plot_PP_cleaned = {'x': [i for i in range(1,l+1)],'y': Effective_Number_of_Bets(l,1,weights,eigenvalues_sorted_PP_cleaned)[1]}
df_plot_PP_cleaned= pd.DataFrame(data_plot_PP_cleaned)
# Create the plot
sns.set_theme()  # Optional: applies the default seaborn theme styling
plt.figure(figsize=(10, 6))  # Set the figure size
barplot = sns.barplot(x='x', y='y', data=df_plot_PP_cleaned, color='red', saturation=0.7)
# Adding the dashed line at y=1
plt.axhline(1, color='black', linestyle='--')
for index, row in df_plot_PP_cleaned.iterrows():
    barplot.text(row['x'] - 1, row['y'], f'{row["y"]:.2f}', color='black', ha='center', va='bottom')
# Set labels (optional)
plt.title('Diversification Distribution Date_PP_cleaned')
plt.xlabel('principal portfolios')
plt.ylabel('diversification distribution')

## 3. Diversification Distribution Date_Tr_cleaned
eigenvalues_sorted_Tr_cleaned=principal_component_decomposition(cleaned_covariance_matrix_dateTr)[2]
Effective_Number_of_Bets_Tr_cleaned=Effective_Number_of_Bets(l,1,weights,eigenvalues_sorted_Tr_cleaned)[0]
data_plot_Tr_cleaned= {'x': [i for i in range(1,l+1)],'y': Effective_Number_of_Bets(l,1,weights,eigenvalues_sorted_Tr_cleaned)[1]}
df_plot_Tr_cleaned= pd.DataFrame(data_plot_Tr_cleaned)
# Create the plot
sns.set_theme()  # Optional: applies the default seaborn theme styling
plt.figure(figsize=(10, 6))  # Set the figure size
barplot = sns.barplot(x='x', y='y', data=df_plot_Tr_cleaned, color='red', saturation=0.7)
# Adding the dashed line at y=1
plt.axhline(1, color='black', linestyle='--')
for index, row in df_plot_Tr_cleaned.iterrows():
    barplot.text(row['x'] - 1, row['y'], f'{row["y"]:.2f}', color='black', ha='center', va='bottom')
# Set labels (optional)
plt.title('Diversification Distribution Date_Tr_cleaned')
plt.xlabel('principal portfolios')
plt.ylabel('diversification distribution')


plt.show()
print('Effective_Number_of_Bets_PP_raw =',Effective_Number_of_Bets_PP_raw)
print('Effective_Number_of_Bets_Tr_raw =',Effective_Number_of_Bets_Tr_raw)
print('Effective_Number_of_Bets_PP_cleaned =',Effective_Number_of_Bets_PP_cleaned)
print('Effective_Number_of_Bets_Tr_cleaned =',Effective_Number_of_Bets_Tr_cleaned)

### Question 2.f) 

In [ ]:
def rank_by_values(data):
    # Convert the data to a pandas Series
    crypto_returns = pd.Series(data)
    
    # Create a DataFrame with the value and calculate the rank
    crypto_with_values = crypto_returns.to_frame(name='Value')
    crypto_with_values['Rank'] = crypto_with_values['Value'].rank(method='average', ascending=False)
    
    # Sort by rank
    sorted_crypto_with_values = crypto_with_values.sort_values(by='Rank')
    
    # Reset index to show the name as a column
    sorted_crypto_with_values.reset_index(inplace=True)
    sorted_crypto_with_values.rename(columns={'index': 'Name'}, inplace=True)
    
    return sorted_crypto_with_values[['Rank', 'Name', 'Value']]

from scipy.stats import kendalltau
def kendall_tau_correlation_matrix(data):
    n = data.shape[1]
    corr_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                corr_matrix[i, j] = 1
            elif i < j:
                tau, _ = kendalltau(data[:, i], data[:, j])
                corr_matrix[i, j] = tau
                corr_matrix[j, i] = tau
    return corr_matrix


In [ ]:
asset=['ADA','BCH' ,'BTC','DOGE', 'ETH','LINK','LTC','MANA','XLM','XRP','SPXT','XCMP','USSOC','VIX']
euler_rc_date_Tr_raw_df=pd.DataFrame({'Name':asset,'risk':euler_rc_date_Tr_raw})
euler_rc_date_Tr_cleaned_df=pd.DataFrame({'Name':asset,'risk':euler_rc_date_Tr_cleaned})

df_TR_raw=rank_by_values(euler_rc_date_Tr_raw_df.reset_index(drop=True).set_index('Name')['risk'])
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='Name', y='Value', data=df_TR_raw,color='red')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
plt.title('Values of risk contribution dateTr(raw)')
plt.tight_layout() 

df_TR_cleaned=rank_by_values(euler_rc_date_Tr_cleaned_df.reset_index(drop=True).set_index('Name')['risk'])
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='Name', y='Value', data=df_TR_cleaned,color='red')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
plt.title('Values of risk contribution dateTr(cleaned)')
plt.tight_layout() 

df=rank_by_values(portfolio_EW_log.loc[dateTr][:-1])
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='Name', y='Value', data=df,color='red')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
plt.title('Return losses per asset')
plt.show()



In [ ]:
#Kendall's Tau rank order correlation matrix 

data_raw = np.array([portfolio_EW_log.loc[dateTr][:-1].tolist(),euler_rc_date_Tr_raw_df.reset_index(drop=True).set_index('Name')['risk'].tolist()]).T  # Transpose to align lists as columns
# Compute the Kendall's Tau correlation matrix
corr_matrix = kendall_tau_correlation_matrix(data_raw)
# Plot the correlation matrix
plt.figure(figsize=(8, 6))
plt.imshow(corr_matrix, interpolation='none', cmap='coolwarm', aspect='auto')
plt.colorbar()
plt.xticks(range(data_raw.shape[1]), ['portfolio_EW', 'euler_rc_date_Tr_raw'])
plt.yticks(range(data_raw.shape[1]), ['portfolio_EW', 'euler_rc_date_Tr_raw'])
plt.title('Kendall’s Tau Correlation Matrix_raw')


data_cleanedKD = np.array([portfolio_EW_log.loc[dateTr][:-1].tolist(),euler_rc_date_Tr_cleaned_df.reset_index(drop=True).set_index('Name')['risk'].tolist()]).T  # Transpose to align lists as columns
# Compute the Kendall's Tau correlation matrix
corr_matrix = kendall_tau_correlation_matrix(data_cleanedKD)
# Plot the correlation matrix
plt.figure(figsize=(8, 6))
plt.imshow(corr_matrix, interpolation='none', cmap='coolwarm', aspect='auto')
plt.colorbar()
plt.xticks(range(data_raw.shape[1]), ['portfolio_EW', 'euler_rc_date_cleaned_raw'])
plt.yticks(range(data_raw.shape[1]), ['portfolio_EW', 'euler_rc_date_cleaned_raw'])
plt.title('Kendall’s Tau Correlation Matrix_cleaned')

plt.show()



## Part 3 Risk based portfolios

In [ ]:
VAR_COV_PP = cleaned_covariance_matrix_datePP_df
VAR_COV_TR = cleaned_covariance_matrix_dateTr_df
RETURNS_PP = data_cleaned.loc[:datePP]
RETURNS_TR = data_cleaned.loc[datePP:dateTr]

Create some useful functions for the section

In [ ]:
def get_return_portfolio(weights, returns):
    """Computes the return of the portfolio.

    Args:
        weights (dict): weigths of each assets in the portfolio: {asset: weight}
        returns (dataframe): matrix of returns
    """
    key = list(weights.keys())
    col = returns.columns.to_list()

    for i in range(len(weights)): # make sure columns match
        if not key[i] == col[i]:
            raise ValueError(f"There is an issue with the order of the keys of the weights and the data. They must have the same sequence. Weights: {key}, dataframe: {col}")
    
    mean_vec = returns.mean()
    
    ret = sum([weights[stock] * mean_vec[stock] for stock in weights.keys()])
    
    return ret

In [ ]:
def get_variance_portfolio(weights, covariance):
    """Computes the variance of the portfolio.

    Args:
        weights (dict): weigths of each assets in the portfolio: {asset: weight}
        covariance (dataframe): matrix of variance-covariance
    """
    key = list(weights.keys())
    col = covariance.columns.to_list()

    for i in range(len(weights)): # make sure columns match
        if not key[i] == col[i]:
            raise ValueError(f"There is an issue with the order of the keys of the weights and the data. They must have the same sequence. Weights: {key}, dataframe: {col}")
    
    values = np.array(list(weights.values()))
    
    var = values@covariance@values.T
    
    return var

In [ ]:
def disp_portfolio_weights(weights, title=None):
    """Displays the weights from a dictionnary"""
    if title:
        print(title)
        for (k,v) in enumerate(weights):
            print(f"- {v}\t: {round(weights[v], 4)}")

def save_portfolio_weights(weights, title, date, rounding = 4):
    weights_df = pd.DataFrame.from_dict(weights, orient='index', columns=['Weights'])
    weights_df.Weights = weights_df.Weights.apply(lambda row: str(round(row, rounding)))
    weights_df.to_latex(f"{title}_{date}.tex")

### Question 3.a) Portfolio creation

#### Question 3.a) i. Minimum variance portfolio

In [ ]:
# Function to calculate the portfolio variance
def portfolio_variance(weights, covariance_matrix):
    return weights.T @ covariance_matrix @ weights

# Number of assets
num_assets = VAR_COV_PP.shape[0]  

# Constraints for the optimization (weights sum to 1 and non-negative weights)
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # The weights must sum to 1
bounds = tuple((0, 1) for _ in range(num_assets))               # Non-negative weights for each asset

# Initial guess (equal weights)
initial_weights = np.ones(num_assets) / num_assets

# Portfolio optimization for datePP
result_datePP = minimize(portfolio_variance, initial_weights, args=(VAR_COV_PP,),
                         method='SLSQP', bounds=bounds, constraints=constraints)
weights_datePP_min_var = result_datePP.x
weights_datePP_min_var_dic = {VAR_COV_PP.columns.to_list()[i]:result_datePP.x[i] for i in range(num_assets)}

# Portfolio optimization for dateTr
result_dateTr = minimize(portfolio_variance, initial_weights, args=(VAR_COV_TR,),
                         method='SLSQP', bounds=bounds, constraints=constraints)
weights_dateTr_min_var = result_dateTr.x

weights_dateTr_min_var_dic = {VAR_COV_PP.columns.to_list()[i]:result_dateTr.x[i] for i in range(num_assets)}


# Display results
# print("Minimum Variance Portfolio Weights at DatePP:", weights_datePP_min_var)
# print("Minimum Variance Portfolio Weights at DateTr:", weights_dateTr_min_var)

save_portfolio_weights(weights_datePP_min_var_dic, "tables/part3ai_weights_min_var", "datePP")
save_portfolio_weights(weights_dateTr_min_var_dic, "tables/part3ai_weights_min_var", "dateTr")

In [ ]:
disp_portfolio_weights(weights_datePP_min_var_dic, "Minimum Variance Portfolio Weights at DatePP:")

In [ ]:
disp_portfolio_weights(weights_dateTr_min_var_dic, "Minimum Variance Portfolio Weights at DateTr:")

In [ ]:
# Compute return and variances of the 2 portfolios
return_min_var_datePP = get_return_portfolio(weights_datePP_min_var_dic, RETURNS_TR)
return_min_var_dateTr = get_return_portfolio(weights_dateTr_min_var_dic, RETURNS_TR)
var_min_var_datePP =    get_variance_portfolio(weights_datePP_min_var_dic, VAR_COV_PP)
var_min_var_dateTr =    get_variance_portfolio(weights_dateTr_min_var_dic, VAR_COV_TR)

print(f"Minimum variance portfolio as of datePP")
print(f"  - Expected return: \t {round(return_min_var_datePP, 4)}%")
print(f"  - Volatility: \t {round(var_min_var_datePP ** 0.5, 4)}%")
print()
print(f"Minimum variance portfolio as of dateTr")
print(f"  - Expected return: \t {round(return_min_var_dateTr, 4)}%")
print(f"  - Volatility: \t {round(var_min_var_dateTr ** 0.5, 4)}%")

#### Question 3.a) ii. Equal risk contribution

In [ ]:
# Risk contribution function
def risk_contribution(weights, covariance_matrix):
    total_portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    marginal_contributions = np.dot(covariance_matrix, weights)
    risk_contributions = weights * marginal_contributions / total_portfolio_variance
    return risk_contributions

def objective_function(weights, covariance_matrix):
    target_risk_contribution = np.ones(num_assets) / num_assets
    actual_risk_contributions = risk_contribution(weights, covariance_matrix)
    return np.sum((actual_risk_contributions - target_risk_contribution) ** 2)

# Portfolio optimization for datePP
result_erc_datePP = minimize(objective_function, initial_weights, args=(VAR_COV_PP,),
                             method='SLSQP', bounds=bounds, constraints=constraints)
weights_erc_datePP = result_erc_datePP.x

# Weights to dict
weights_erc_datePP_dic = {VAR_COV_PP.columns.to_list()[i]:result_erc_datePP.x[i] for i in range(num_assets)}

# Portfolio optimization for dateTr
result_erc_dateTr = minimize(objective_function, initial_weights, args=(VAR_COV_TR,),
                             method='SLSQP', bounds=bounds, constraints=constraints)
weights_erc_dateTr = result_erc_dateTr.x

# Weights to dict
weights_erc_dateTr_dic = {VAR_COV_PP.columns.to_list()[i]:result_erc_dateTr.x[i] for i in range(num_assets)}

# Display results
# print("Equal Risk Contribution Portfolio Weights at DatePP:\n", weights_erc_datePP)
# print("\nEqual Risk Contribution Portfolio Weights at DateTr:\n", weights_erc_dateTr)

save_portfolio_weights(weights_erc_datePP_dic, "tables/part3aii_weights_erc", "datePP")
save_portfolio_weights(weights_erc_dateTr_dic, "tables/part3aii_weights_erc", "dateTr")


In [ ]:
disp_portfolio_weights(weights_erc_datePP_dic, "Equal Risk Contribution Portfolio Weights at DatePP:")

In [ ]:
disp_portfolio_weights(weights_erc_dateTr_dic, "Equal Risk Contribution Portfolio Weights at DateTr:")

In [ ]:
# Compute return and variances of the 2 portfolios
return_erc_datePP = get_return_portfolio(weights_erc_datePP_dic, RETURNS_TR)
return_erc_dateTr = get_return_portfolio(weights_erc_dateTr_dic, RETURNS_TR)
var_erc_datePP =    get_variance_portfolio(weights_erc_datePP_dic, VAR_COV_PP)
var_erc_dateTr =    get_variance_portfolio(weights_erc_dateTr_dic, VAR_COV_TR)

print(f"Equal Risk Contribution portfolio as of datePP")
print(f"  - Expected return: \t {round(return_erc_datePP, 4)}%")
print(f"  - Volatility: \t {round(return_erc_dateTr ** 0.5, 4)}%")
print()
print(f"Equal Risk Contribution portfolio as of dateTr")
print(f"  - Expected return: \t {round(var_erc_datePP, 4)}%")
print(f"  - Volatility: \t {round(var_erc_dateTr ** 0.5, 4)}%")

#### Question 3.a) iii. Minimum effective number of bets portfolio

In [ ]:
#S is the Covariance Matrix 
#A is the Conditional Matrix
#E is the Matrix of the eigenvectors
#L is the List of the eigenvalues

def real_parts(complex_list):
    return [z.real for z in complex_list]

def gen_first_eig_vect(S, A):
    N = S.shape[0]
    P = np.eye(N)
    if np.linalg.matrix_rank(A) > 0:
        P = np.eye(N) - A.T @ np.linalg.inv(A @ A.T) @ A
    L, E = np.linalg.eig(P @ S @ P.T)
    
    indexx = np.argmax(L)
    e = E[:, indexx]

    return e


def gen_pcbasis(S, A):      #Compute the conditional principal portfolios.
    if A.size == 0:
        N = S.shape[0]
        K = 0
        E_, L_ = np.linalg.eig(S)
        idx = np.argsort(-np.diag(L_))
        E = E_[:, idx]
        L = np.diag(L_)[idx]
    else:
        K, N = A.shape
        E = np.empty((N, 0))
        B = A
        for n in range(N-K):
            if E.shape[1] > 0:
                B = np.vstack([A, E.T @ S])
            e = gen_first_eig_vect(S, B)
            E = np.hstack([E, e[:, np.newaxis]])

        for n in range(N-K, N):
            B = E.T @ S
            e = gen_first_eig_vect(S, B)
            E = np.hstack([E, e[:, np.newaxis]])

        # Swap order
        E = np.hstack([E[:, N-K:], E[:, :N-K]])

    L = np.diag(E.T @ S @ E)
    G = np.diag(np.sqrt(L)) @ np.linalg.inv(E)
    if K > 0:
        G = G[K:, :]

    return E, L, G

def diversification_distribution_conditional(weights,eigenvalues,k):
    var=0
    l=len(eigenvalues)
    List_probability_distribution=[]
    for i in range(k,l):
        var+=(weights[i]**2)*eigenvalues[i]
    for i in range(k,l):
        List_probability_distribution.append((weights[i]**2)*eigenvalues[i]/var)
    return(List_probability_distribution)

def Effective_Number_of_Bets_conditional(K,E,weights,eigenvalues):
    S=0
    weights=np.linalg.inv(E)@weights
    L=diversification_distribution_conditional(weights,eigenvalues,K)
    N=len(L)
    for i in range(N):
        S+=L[i]*np.log(L[i])
    return(np.exp(-S))


In [ ]:
E,L,G=gen_pcbasis(VAR_COV_TR,np.ones((1,l)))
N=len(L)
def constraint(weights):
    return np.sum(weights) - 1

def objective_function(weights):
    return(Effective_Number_of_Bets_conditional(1,E,weights,L))

# Initial guess
initial_guess = [1/N for i in range(N)]

# Bounds for each weight (non-negativity constraint)
bounds = [(0, None)] * N

# Constraints
constraints = [{'type': 'eq', 'fun': constraint}]

# Minimize entropy subject to constraints
result = minimize(objective_function, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)


# Dictionary of weights
weights_minimum_effective_number_of_bets_TR={}
for i in range(14):
    weights_minimum_effective_number_of_bets_TR[asset[i]]=result.x[i]

E,L,G=gen_pcbasis(VAR_COV_PP,np.ones((1,l)))
N=len(L)
def constraint(weights):
    return np.sum(weights) - 1

def objective_function(weights):
    return(Effective_Number_of_Bets_conditional(1,E,weights,L))

# Initial guess
initial_guess = [1/N for i in range(N)]

# Bounds for each weight (non-negativity constraint)
bounds = [(0, None)] * N

# Constraints
constraints = [{'type': 'eq', 'fun': constraint}]

# Minimize entropy subject to constraints
result = minimize(objective_function, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)

# Dictionary of weights
weights_minimum_effective_number_of_bets_PP={}
for i in range(14):
    weights_minimum_effective_number_of_bets_PP[asset[i]]=result.x[i]

# Save the weights
save_portfolio_weights(weights_minimum_effective_number_of_bets_PP, "tables/part3aiii_weights_min_eff_nb_bets", "datePP")
save_portfolio_weights(weights_minimum_effective_number_of_bets_TR, "tables/part3aiii_weights_min_eff_nb_bets", "dateTr")

In [ ]:
disp_portfolio_weights(weights_minimum_effective_number_of_bets_PP, "Minimum effecitve number of bets portfolio Weights at DatePP:")

In [ ]:
disp_portfolio_weights(weights_minimum_effective_number_of_bets_TR, "Minimum effecitve number of bets portfolio Weights at DateTr:")

In [ ]:
# Compute return and variances of the 2 portfolios
return_minimum_effective_number_of_bets_PP = get_return_portfolio(weights_minimum_effective_number_of_bets_PP, RETURNS_PP)
return_minimum_effective_number_of_bets_Tr = get_return_portfolio(weights_minimum_effective_number_of_bets_TR, RETURNS_TR)
variance_minimum_effective_number_of_bets_PP =    get_variance_portfolio(weights_minimum_effective_number_of_bets_PP, VAR_COV_PP)
variance_minimum_effective_number_of_bets_Tr =    get_variance_portfolio(weights_minimum_effective_number_of_bets_TR, VAR_COV_TR)

print(f"Equal Risk Contribution portfolio as of datePP")
print(f"  - Expected return: \t {round(return_minimum_effective_number_of_bets_PP, 5)}%")
print(f"  - Volatility: \t {round(variance_minimum_effective_number_of_bets_PP ** 0.5, 4)}%")
print()
print(f"Equal Risk Contribution portfolio as of dateTr")
print(f"  - Expected return: \t {round(return_minimum_effective_number_of_bets_Tr, 5)}%")
print(f"  - Volatility: \t {round(variance_minimum_effective_number_of_bets_Tr ** 0.5, 4)}%")

#### Question 3.a) iv. Hierarchical risk portolio

In [ ]:
from HRP_portolio import *

def get_HRP(data, covariance, show = None):

    # Compute distance matrix
    volatilities = np.sqrt(np.diag(covariance))
    correlation = covariance / np.outer(volatilities, volatilities)
    correlation = data.corr() # comment when correlation matrix is symmetric
    distance_matrix = correlation.apply(lambda row: (0.5*(1-row))**0.5)

    # Get ordered distance matrix, cluster order and linkage matrix (uses gmarti.gitlab.io)
    ordered_dist_mat, res_order, res_linkage = compute_serial_matrix(distance_matrix.values, method='single')

    # Formatting of the dataframe for clearer representation
    ordered_dist_mat_df = pd.DataFrame(
        ordered_dist_mat, 
        index = list(correlation.columns), 
        columns=list(correlation.columns)

    )

    res_linkage_df = pd.DataFrame(
        res_linkage, 
        index = [f"Cluster {i+1}" for i in range(res_linkage.shape[0])], 
        columns= ["Elem 1", "Elem 2", "Distance", "Original nb elem cluster"]
    )

    if show:
        print("\nOrdered distance matrix")
        display(ordered_dist_mat_df)

        print("\nLinkage matrix")
        display(res_linkage_df)

        print("\nRes order")
        print(res_order)
        

    return ordered_dist_mat_df, res_linkage_df, distance_matrix, res_order

In [ ]:
def show_cluster_HRP(res_linkage, save=False):
    """ Displays the dendrogram of the clusters of the assets
        Input: res_linkage
    """
    scipy.cluster.hierarchy.dendrogram(res_linkage)
    if save:
        plt.savefig("figures/part3aiv_dendrogram_HRP")

    plt.title("Hierarchical cluster of the assets")
    plt.show()

# show_cluster_HRP(res_linkage_df)

In [ ]:
def get_HRP_weights(covariance, res_order, show = False):

    # The input matrice needs to have numerical indices and column names
    c = covariance.copy()
    c.reset_index(drop=True, inplace = True)
    c.columns = list(c.index) 

    HRP_weights = compute_HRP_weights(
        c,
        res_order
    ).sort_index()

    HRP_weights_dic = {
        list(covariance.columns)[i]:HRP_weights.loc[i] for i in range(len(HRP_weights))
        }

    if show:

        disp_portfolio_weights(HRP_weights_dic, title="Hierarchical risk portfolio weights:")
        
        # print("Hierarchical risk portfolio weights:")
        # for (k,v) in enumerate(HRP_weights_dic):
        #     print(f"- {v}\t: {round(HRP_weights_dic[v], 4)}")

    return HRP_weights_dic

    
# w = get_HRP_weights(VAR_COV_PP, res_order, show=True)#covariance_matrix

In [ ]:
def HRP(data, covariance, show=False):

    ordered_dist_mat_df, res_linkage_df, distance_matrix, res_order = get_HRP(data=data, covariance = covariance, show=show)
    
    if show:
        show_cluster_HRP(res_linkage_df)

    weights = get_HRP_weights(covariance = covariance, res_order = res_order, show = show)

    return weights, ordered_dist_mat_df, res_linkage_df, distance_matrix, res_order

# weights = HRP(data = DATA_LOG_RETURN, covariance = VAR_COV_PP, show = True)[0]

In [ ]:
# Last function that allows to display all results of HRP for a given date, and saving tables
def run_HRP(data, covariance, date, save=False, rounding=4, show=False):

    if show:
        print(f"-- Hierarchical risk parity portfolio results for date {date} --")

    # Run the HRP function to get all the informations
    weights, ordered_dist_mat_df, res_linkage_df, distance_matrix, res_order = HRP(data = data, covariance = covariance, show = show)

    if save:

        # Save weights
        HRP_weights_df_latex = pd.DataFrame.from_dict(weights, orient='index', columns=['Weights'])
        HRP_weights_df_latex.Weights = HRP_weights_df_latex.Weights.apply(lambda row: str(round(row, rounding)))
        HRP_weights_df_latex.to_latex(f"tables/part3aiv_HRP_weights_{date}.tex")

        # Save ordered_dist_mat_df
        ordered_dist_mat_df = ordered_dist_mat_df.round(rounding).to_latex(f"tables/part3aiv_HRP_ordered_distance_matrix_{date}.tex")

        # Save distance_matrix
        distance_matrix = distance_matrix.round(rounding).to_latex(f"tables/part3aiv_HRP_distance_matrix_{date}.tex")

        # Save res_linkage_df
        res_linkage_df["Elem 1"] = res_linkage_df["Elem 1"].astype(int)
        res_linkage_df["Elem 2"] = res_linkage_df["Elem 2"].astype(int)
        res_linkage_df["Original nb elem cluster"] = res_linkage_df["Original nb elem cluster"].astype(int)
        res_linkage_df["Distance"] = res_linkage_df["Distance"].round(rounding)
        res_linkage_df = res_linkage_df.to_latex(f"tables/part3aiv_HRP_res_linkage_{date}.tex")


    return weights, ordered_dist_mat_df, res_linkage_df, distance_matrix, res_order

# run_HRP(data = DATA_LOG_RETURN, covariance = VAR_COV_PP, date = "datePP", show = False)

In [ ]:
# Run the HRP on both dates

# Data for datePP and dateTr
HRP_data_datePP = DATA_LOG_RETURN[DATA_LOG_RETURN.index < datePP]
HRP_data_dateTr = DATA_LOG_RETURN[DATA_LOG_RETURN.index < dateTr]
HRP_data_dateTr = HRP_data_dateTr[HRP_data_dateTr.index > datePP]

# As of datePP
weights_HRP_datePP_dic, ordered_dist_mat_df_datePP, res_linkage_df_datePP, distance_matrix_datePP, res_order_datePP = run_HRP(data = HRP_data_datePP, covariance = VAR_COV_PP, date = "datePP", show = True, save = True)
print("\n")

# As of dateTr
weights_HRP_dateTr_dic, ordered_dist_mat_df_dateTr, res_linkage_df_dateTr, distance_matrix_dateTr, res_order_dateTr = run_HRP(data = HRP_data_dateTr, covariance = VAR_COV_TR, date = "dateTr", show = True, save = True)

#weights_datePP, ordered_dist_mat_df_datePP, res_linkage_df_datePP, distance_matrix_datePP, res_order_datePP = HRP(data = DATA_LOG_RETURN, covariance = VAR_COV_PP, show = True)

# weights_datePP, ordered_dist_mat_df_datePP, res_linkage_df_datePP, distance_matrix_datePP, res_order_datePP = run_HRP(data = DATA_LOG_RETURN, covariance = VAR_COV_PP, date = "datePP", show = True, save = True)

In [ ]:
# Compute return and variances of the 2 portfolios
return_HRP_datePP = get_return_portfolio(weights_HRP_datePP_dic, RETURNS_PP)
return_HRP_dateTr = get_return_portfolio(weights_HRP_dateTr_dic, RETURNS_TR)
var_HRP_datePP =    get_variance_portfolio(weights_HRP_datePP_dic, VAR_COV_PP)
var_HRP_dateTr =    get_variance_portfolio(weights_HRP_dateTr_dic, VAR_COV_TR)

print(f"HRP portfolio as of datePP")
print(f"  - Expected return: \t {round(return_HRP_datePP, 5)}%")
print(f"  - Volatility: \t {round(var_HRP_datePP ** 0.5, 4)}%")
print()
print(f"HRP portfolio as of dateTr")
print(f"  - Expected return: \t {round(return_HRP_dateTr, 5)}%")
print(f"  - Volatility: \t {round(var_HRP_dateTr ** 0.5, 4)}%")

#### Plot distance matrix - Evidence of seriation

In [ ]:
# Toy example to show the seriation of the distance matrix
correlation = HRP_data_datePP.corr()
distance_matrix = correlation.apply(lambda row: (0.5*(1-row))**0.5)

ordered_dist_mat, res_order, res_linkage = compute_serial_matrix(distance_matrix.values, method='single')

plt.figure(figsize=(8, 5))
sns.heatmap(distance_matrix, cmap='coolwarm')
plt.savefig("figures/part3aiv_distance_matrix")
plt.title('Distance matrix')
plt.show()

plt.figure(figsize=(8, 5))
sns.heatmap(ordered_dist_mat,cmap='coolwarm')
plt.savefig("figures/part3aiv_ordered_distance_matrix")
plt.title('Ordered distance matrix')
plt.show()

## Part 4 Extensions to Hierarchical Risk Parity

## Robust Nonparametric Copula Based Dependence Estimators

$C_N(\frac{i}{N}, \frac{j}{N}) = \frac{1}{N} [\# \text{ of } (X_k^1, X_k^2) \text{ st } X_k^1 <= X_i^1 \text{ and } X_k^2<= X_i^2]$

In [ ]:
# Function to compute the empirical CDF using rank data
def empirical_cdf(x):
    return x.rank(method='max').values / len(x)

def get_SW_sigma_estimator(df):

    # Number of observations
    N = len(df)

    # Get the list of variable names
    variables = df.columns

    # Initialize the matrix to store the SW estimators
    sigma_matrix = pd.DataFrame(index=variables, columns=variables)

    # Compute the empirical CDFs for all variables and convert to numpy array
    cdf_data = df.apply(empirical_cdf).to_numpy()

    # Calculate the Schweizer-Wolff sigma estimator for all pairs of variables
    for i in tqdm(range(len(variables))):
        for j in range(i, len(variables)):
            if i == j:
                sigma_matrix.iloc[i, j] = 0
            else:
                # Get empirical CDFs for the pair of variables
                F1 = cdf_data[:, i]
                F2 = cdf_data[:, j]

                # Construct the empirical copula matrix
                empirical_copula = np.zeros((N, N))
                for k in range(N):
                    empirical_copula[k] = np.mean((F1[:, None] <= (k + 1) / N) & (F2[:, None] <= (np.arange(N) + 1) / N), axis=0)

                # Calculate the Schweizer-Wolff sigma estimator
                outer_product = np.outer(np.arange(1, N + 1) / N, np.arange(1, N + 1) / N)
                sigma = 12 / (N ** 2 - 1) * np.sum(np.abs(empirical_copula - outer_product))
                sigma_matrix.iloc[i, j] = sigma
                sigma_matrix.iloc[j, i] = sigma
    
    return sigma_matrix

In [ ]:
from HRP_portolio import *

# Last function that allows to display all results of HRP for a given date, and saving tables
def run_HRP_SW(data, sigmaSW, date, save=False, rounding=4):
    # Get the CDF
    cdf_data = data.apply(empirical_cdf).to_numpy()
    
    # Run the HRP function to get all the informations
    seriated_dist, res_order, res_linkage = compute_serial_matrix(sigmaSW.values)
    HRP_SW_weights = compute_HRP_weights(pd.DataFrame(cdf_data), res_order)
    HRP_SW_weights.sort_index()

    HRP_SW_weights = {list(data)[i]:HRP_SW_weights.loc[i] for i in range(len(HRP_SW_weights))}
    
    #weights, ordered_dist_mat_df, res_linkage_df, distance_matrix, res_order = HRP(data = data, covariance = sigmaSW, show = show)

    ordered_dist_mat_df = pd.DataFrame(
            seriated_dist, 
            index = list(data.columns), 
            columns=list(data.columns)   
        )

    if save:

        # Save weights
        HRP_SW_weights_df_latex = pd.DataFrame.from_dict(HRP_SW_weights, orient='index', columns=['Weights'])
        HRP_SW_weights_df_latex.Weights = HRP_SW_weights_df_latex.Weights.apply(lambda row: str(round(row, rounding)))
        HRP_SW_weights_df_latex.to_latex(f"tables/part4_HRP_SW_weights_{date}.tex")

        # Save ordered_dist_mat_df
        ordered_dist_mat_df = ordered_dist_mat_df.round(rounding).to_latex(f"tables/part4_HRP_SW_ordered_distance_matrix_{date}.tex")

        res_linkage_df = pd.DataFrame(
            res_linkage, 
            index = [f"Cluster {i+1}" for i in range(res_linkage.shape[0])], 
            columns= ["Elem 1", "Elem 2", "Distance", "Original nb elem cluster"]
        )
        res_linkage_df["Elem 1"] = res_linkage_df["Elem 1"].astype(int)
        res_linkage_df["Elem 2"] = res_linkage_df["Elem 2"].astype(int)
        res_linkage_df["Original nb elem cluster"] = res_linkage_df["Original nb elem cluster"].astype(int)
        res_linkage_df["Distance"] = res_linkage_df["Distance"].round(rounding)
        res_linkage_df = res_linkage_df.to_latex(f"tables/part4_HRP_SW_res_linkage_{date}.tex")


    return HRP_SW_weights, ordered_dist_mat_df


In [ ]:
sigma_Tr = get_SW_sigma_estimator(RETURNS_TR)
sigma_PP = get_SW_sigma_estimator(RETURNS_PP)

# sigma_Tr.to_csv("sigma_Tr.csv")
# sigma_PP.to_csv("sigma_PP.csv")

In [77]:
# sigma_Tr = pd.read_csv("sigma_Tr.csv", index_col=[0])
# sigma_PP = pd.read_csv("sigma_PP.csv", index_col=[0])

weights_HRP_SW_datePP_dic, seriated_dist_Tr = run_HRP_SW(RETURNS_TR, sigma_Tr, "dateTr")
weights_HRP_SW_dateTr_dic, seriated_dist_PP = run_HRP_SW(RETURNS_TR, sigma_Tr, "datePP")

disp_portfolio_weights(weights_HRP_SW_datePP_dic, title="HRP SW date PP")
disp_portfolio_weights(weights_HRP_SW_dateTr_dic, title="\nHRP SW date Tr")

HRP SW date PP
- ADA	: 0.0078
- BCH	: 0.0106
- BTC	: 0.0376
- DOGE	: 0.0194
- ETH	: 0.011
- LINK	: 0.0256
- LTC	: 0.0427
- MANA	: 0.0157
- XLM	: 0.038
- XRP	: 0.6566
- SPXT	: 0.0441
- XCMP	: 0.022
- USSOC	: 0.0245
- VIX	: 0.0445

HRP SW date Tr
- ADA	: 0.0078
- BCH	: 0.0106
- BTC	: 0.0376
- DOGE	: 0.0194
- ETH	: 0.011
- LINK	: 0.0256
- LTC	: 0.0427
- MANA	: 0.0157
- XLM	: 0.038
- XRP	: 0.6566
- SPXT	: 0.0441
- XCMP	: 0.022
- USSOC	: 0.0245
- VIX	: 0.0445


In [ ]:
# Compute return and variances of the 2 portfolios
return_HRP_SW_datePP = get_return_portfolio(weights_HRP_SW_datePP_dic, RETURNS_PP)
return_HRP_SW_dateTr = get_return_portfolio(weights_HRP_SW_dateTr_dic, RETURNS_TR)
var_HRP_SW_datePP =    get_variance_portfolio(weights_HRP_SW_datePP_dic, VAR_COV_PP)
var_HRP_SW_dateTr =    get_variance_portfolio(weights_HRP_SW_dateTr_dic, VAR_COV_TR)

print(f"Equal Risk Contribution portfolio as of datePP")
print(f"  - Expected return: \t {round(return_HRP_SW_datePP, 5)}%")
print(f"  - Volatility: \t {round(var_HRP_SW_datePP ** 0.5, 4)}%")
print()
print(f"Equal Risk Contribution portfolio as of dateTr")
print(f"  - Expected return: \t {round(return_HRP_SW_dateTr, 5)}%")
print(f"  - Volatility: \t {round(var_HRP_SW_dateTr ** 0.5, 4)}%")

In [ ]:
# sigma_Tr.to_csv("sigma_Tr.csv")
# sigma_PP.to_csv("sigma_PP.csv")
